In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as soup

In [2]:
final_data = pd.DataFrame()

In [3]:
for i in range(1, 29) :
    
    header = {'Origin': 'https://www.1mg.com',
             'Referer': 'https://www.1mg.com/categories/diabetes/diabetic-medicines-583?filter=true&uses=Diabetes%20Care&pageNumber={}'.format(i),
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
             }
    
    html = requests.get('https://www.1mg.com/categories/diabetes/diabetic-medicines-583?filter=true&uses=Diabetes%20Care&pageNumber={}'.format(i), headers=header)
    soup_obj = soup(html.content, 'lxml')
    
    product_name = []
    quantity = []
    product_rating = []
    product_rating_count = []
    product_price = []

    for item in soup_obj.findAll('div', class_ = 'col-md-3 col-sm-4 col-xs-6 style__container___1TL2R') :

        for p_name in item.findAll('div', class_ = 'style__pro-title___2QwJy') :
            product_name.append(p_name.text.strip())

        for p_quan in item.findAll('div', class_ = 'style__pack-size___2JQG7') :
            quantity.append(p_quan.text.strip())

        curr_rating = []
        for p_rate in item.findAll('div', class_ = 'CardRatingDetail__ratings-container___2ZTSK') :
            curr_rating.append(p_rate.text.strip())
        if len(curr_rating) > 0 :
            product_rating += curr_rating
        else :
            product_rating.append('0.0')

        curr_rating_count = []
        for p_rate_count in item.findAll('span', class_ = 'CardRatingDetail__ratings-header___2yyQW') :
            curr_rating_count.append(p_rate_count.text.strip())
        if len(curr_rating_count) > 0 :
            product_rating_count += curr_rating_count
        else :
            product_rating_count.append('0 ratings')

        for p_price in item.findAll('div', class_ = 'style__price-tag___cOxYc') :
            product_price.append(p_price.text.strip())
            
    temp_price = []
    for pp in product_price :
        temp_price.append(pp.split('₹')[1])

    temp_rating = []
    for rat in product_rating_count :
        temp_rating.append(rat.split(' ')[0])
        
    diction = {'product' : product_name, 'quantity' : quantity, 
               'rating' : product_rating, 'number of ratings' : temp_rating,
               'price' : temp_price}
    
    curr_data = pd.DataFrame(diction)
    
    final_data = final_data.append(curr_data, ignore_index = True)

In [4]:
final_data

,product,quantity,rating,number of ratings,price
0,Bgr 34 Tablet,bottle of 100 tablets,4.2,1082,546
1,1mg Karela Pure Natural Extract 500mg Capsule,bottle of 60 capsules,4.2,205,198
2,Dr. Reckeweg R40 Diabetes Drop Homeopathic Med...,bottle of 22 ml Drop,4.7,81,243.5
3,SBL Syzygium Jambolanum Mother Tincture Q,bottle of 30 ml Mother Tincture,4.6,89,96
4,Dhootapapeshwar Madhumeha Kusumakar Rasa,box of 30 tablets,4.3,126,660
...,...,...,...,...,...
871,Bjain Vaccinium Myrtillus Globules 1000 CH,bottle of 6 gm Globules,0.0,0,85
872,Bjain Gymnema Sylvestre Dilution 3X,bottle of 10 ml Dilution,0.0,0,79
873,Thrillx Herbs Jamun Powder,packet of 100 gm Powder,0.0,0,249
874,Thrillx Herbs Vijaysar Powder,packet of 100 gm Powder,0.0,0,139


In [5]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product            876 non-null    object
 1   quantity           876 non-null    object
 2   rating             876 non-null    object
 3   number of ratings  876 non-null    object
 4   price              876 non-null    object
dtypes: object(5)
memory usage: 34.3+ KB


In [6]:
final_data = final_data.astype({'price' : float, 'rating' : float, 'number of ratings' : int})

In [7]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product            876 non-null    object 
 1   quantity           876 non-null    object 
 2   rating             876 non-null    float64
 3   number of ratings  876 non-null    int32  
 4   price              876 non-null    float64
dtypes: float64(2), int32(1), object(2)
memory usage: 30.9+ KB


In [8]:
final_data.drop_duplicates(subset = ['product'], inplace=True)

In [9]:
final_data

,product,quantity,rating,number of ratings,price
0,Bgr 34 Tablet,bottle of 100 tablets,4.2,1082,546.0
1,1mg Karela Pure Natural Extract 500mg Capsule,bottle of 60 capsules,4.2,205,198.0
2,Dr. Reckeweg R40 Diabetes Drop Homeopathic Med...,bottle of 22 ml Drop,4.7,81,243.5
3,SBL Syzygium Jambolanum Mother Tincture Q,bottle of 30 ml Mother Tincture,4.6,89,96.0
4,Dhootapapeshwar Madhumeha Kusumakar Rasa,box of 30 tablets,4.3,126,660.0
...,...,...,...,...,...
870,Bjain Vaccinium Myrtillus Globules 12 CH,bottle of 6 gm Globules,0.0,0,85.0
871,Bjain Vaccinium Myrtillus Globules 1000 CH,bottle of 6 gm Globules,0.0,0,85.0
873,Thrillx Herbs Jamun Powder,packet of 100 gm Powder,0.0,0,249.0
874,Thrillx Herbs Vijaysar Powder,packet of 100 gm Powder,0.0,0,139.0


In [10]:
# final_data.to_csv('diabetes_medicines__data.csv', index=False)